# Direct Preference Optimization (DPO) at scale with QLoRA

This guide provides a step-by-step workflow for preference fine-tuning the [`Qwen/Qwen2.5-7B-Instruct`](https://huggingface.co/Qwen/Qwen2.5-7B-Instruct) model on a multi-GPU Anyscale cluster. You use LLaMA-Factory as the training framework and `QLoRA` to reduce memory requirements and enable efficient multi-GPU training.

DPO aligns a model with human preferences using pairs of “chosen” and “rejected” responses. Rather than training a separate reward model, DPO directly optimizes the policy to increase the likelihood of preferred outputs and decrease the likelihood of rejected ones.

## Step 1: Set up your environment

### Dependencies
First, ensure your environment has the correct libraries. Start with a pre-built container image and install LLaMA-Factory and DeepSpeed on top of it.

Recommended container image:
```bash
anyscale/ray-llm:2.48.0-py311-cu128
```

Execute the following commands to install the required packages and optional tools for experiment tracking and faster model downloads:

In [1]:
%%bash
# Install the specific version of LLaMA-Factory
pip install -q llamafactory==0.9.3

# (Optional) For visualizing training metrics and logs
pip install -q tensorboard==2.20.0

# (Optional) For lightweight 8-bit and 4-bit optimizers and inference
pip install -q bitsandbytes==0.47.0

# (Optional) For AWQ quantization support
pip install -q autoawq==0.2.9

# (Optional) For accelerated model downloads from Hugging Face
pip install -q hf_transfer==0.1.9

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastmcp 2.14.5 requires pydantic[email]>=2.11.7, but you have pydantic 2.10.6 which is incompatible.
mcp 1.26.0 requires pydantic<3.0.0,>=2.11.0, but you have pydantic 2.10.6 which is incompatible.


Successfully registered `llamafactory` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_a6j8iubw9rqbyigfwk9fut4amk/prj_a8aurpnjjkhushuarbyy4kwkre/workspaces/expwrk_kpm6l9gjz6gdcskt2zb8i3fie6?workspace-tab=dependencies
Successfully registered `tensorboard` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_a6j8iubw9rqbyigfwk9fut4amk/prj_a8aurpnjjkhushuarbyy4kwkre/workspaces/expwrk_kpm6l9gjz6gdcskt2zb8i3fie6?workspace-tab=dependencies
Successfully registered `bitsandbytes` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_a6j8iubw9rqbyigfwk9fut4amk/prj_a8aurpnjjkhushuarbyy4kwkre/workspaces/expwrk_kpm6l9gjz6gdcskt2zb8i3fie6?workspace-tab=dependencies
Successfully registered `autoawq` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_a6j8

### Model and compute resources

| Item | Value |
|------|-------|
| **Base model** | [`Qwen/Qwen2.5-7B-Instruct`](https://huggingface.co/Qwen/Qwen2.5-7B-Instruct) |
| **Workers** | 4 × L4 / A10G |

Compared to SFT, DPO holds two copies of the model (policy and reference), and alignment datasets often use long contexts, so it's the ideal workflow for memory optimization techniques such as **QLoRA**. On 24 GB NVIDIA L4 GPUs, running DPO at FP16 for 7B models generally OOMs without QLoRA.

## Step 2: Prepare the dataset

### Understand the dataset
This tutorial uses [`ultrafeedback.jsonl`](https://huggingface.co/datasets/kaitchup/UltraFeedback-prompt-chosen-rejected), a preference dataset tailored for DPO. Each sample contains one instruction **prompt** and two candidate completions: a **preferred** (`chosen`) response and a **less preferred** (`rejected`) response.

This dataset includes:
- `prompt`: An instruction or question to answer, often multi-sentence, with constraints.
- `chosen`: The response that best follows the instruction.
- `rejected`: A weaker alternative for the same prompt.

**Dataset example**
```json
{
  "prompt": "Paraphrase the given questions to have different wording. Your paraphrased questions should have the same answer as the original question. Try to change the sentence as much as possible using synonyms and/or rearranging the structure of the sentence. The questions are in three domains: presidents, national parks, and dogs. Each question has a keyword indicating its domain. Keywords are \"this national park\", \"this dog breed\", and \"this president\", which will be replaced with the name of an actual president, a national park, or a breed of dog. Hence, in paraphrasing, this keyword should also be used the same way. Do not write questions that compare or involve multiple domains. Do not write open-ended or subjective questions (e.g., questions that can be answered differently by different people.) Make your questions specific and concrete. Your question should have the same type of answer as the original question(e.g., if the question is extractive, the paraphrased question should be extractive as well.)\n\nWhat lakes are in this national park?",

  "rejected": "What bodies of water are located in this national park? \n\nWhich president is commonly known for his efforts to protect natural resources?\n\nWhich president is recognized for their dedication to preserving the environment? \n\nWhat type of dog breed is known for its loyalty and affectionate nature?\n\nWhat breed of dog is renowned for its faithfulness and loving personality?",
  
  "chosen": "Which bodies of water can be found within the borders of this particular national park?"
}
```

### Register the dataset

To specify new datasets that are accessible across Ray worker nodes, you must first add a **`dataset_info.json`** to **[storage shared across nodes](https://docs.anyscale.com/configuration/storage#shared)** such as `/mnt/cluster_storage`. This configuration file acts as a central registry for all your datasets. It maps a custom name to your dataset file location, format, and column structure. 

If you plan to run DPO post-training on the `ultrafeedback` dataset, first complete the setup steps below. Ensure that you place the dataset files in a storage location that all workers can access like a shared mount or object storage. Avoid storing large files on the head node. 

`dataset_info.json`
```json
{
  "my_ultrafeedback": {
    "file_name": "/mnt/cluster_storage/ultrafeedback.jsonl",
    "ranking": true,
    "columns": {
      "prompt": "prompt",
      "chosen": "chosen",
      "rejected": "rejected"
    }
  }
}
```

For a more detailed dataset preparation and formatting guide, see [Choose your data format](https://docs.anyscale.com/llm/fine-tuning/data-preparation#preference-methods).

In [2]:
%%bash
# Make sure all files are accessible to worker nodes
# Create a copy of the data in /mnt/cluster_storage
wget https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/datasets/alpaca/ultrafeedback.jsonl -O /mnt/cluster_storage/ultrafeedback.jsonl
# Create a copy of the dataset registry in /mnt/cluster_storage
cp ../dataset-configs/dataset_info.json /mnt/cluster_storage/

--2026-02-08 09:30:25--  https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/datasets/alpaca/ultrafeedback.jsonl
Resolving anyscale-public-materials.s3.us-west-2.amazonaws.com (anyscale-public-materials.s3.us-west-2.amazonaws.com)... 52.218.179.34, 52.92.206.122, 52.92.251.90, ...
Connecting to anyscale-public-materials.s3.us-west-2.amazonaws.com (anyscale-public-materials.s3.us-west-2.amazonaws.com)|52.218.179.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 291881 (285K) [application/x-www-form-urlencoded]
Saving to: ‘/mnt/cluster_storage/ultrafeedback.jsonl’

     0K .......... .......... .......... .......... .......... 17%  266K 1s
    50K .......... .......... .......... .......... .......... 35%  336K 1s
   100K .......... .......... .......... .......... .......... 52%  456K 0s
   150K .......... .......... .......... .......... .......... 70% 66.5M 0s
   200K .......... .......... .......... .......... .........

## Step 3: Create the preference-tuning config (DPO and QLoRA)

Next, create the YAML configuration file that defines your DPO run. It specifies the base model, quantization (QLoRA), dataset, DPO hyperparameters, logging, and Ray cluster resources.

**Important notes:**
- **QLoRA quantization:** `quantization_bit: 4` with `quantization_method: bnb` applies quantization using bitsandbytes, reducing memory while preserving quality. If you use a model *pre-quantized* with AWQ, **omit** these keys.
- **LoRA setup**: If you prefer standard LoRA, **disable quantization** by removing both `quantization_bit` and `quantization_method` from the config.
- **Access & paths:** The YAML only needs to be on the **head node**, but any referenced paths (`dataset_dir`, `output_dir`) must reside on storage **reachable by all workers** (for example, `/mnt/cluster_storage/`).
- **Gated models:** If your base model has gated access (for example, Llama) on Hugging Face, set `HF_TOKEN` in the runtime environment.

### Configure LLaMA-Factory with Ray

**Note**: To customize the training configuration, edit `train-configs/dpo_qlora.yaml`. 

```yaml
# dpo_qlora.yaml

### model
trust_remote_code: true
model_name_or_path: Qwen/Qwen2.5-7B-Instruct

### method
# If you instead want to use just LoRA, or a pre-quantized model like Qwen/Qwen2.5-7B-Instruct-AWQ, then omit the quantization_bit/method keys below
quantization_bit: 4 # 4-bit base weights (QLoRA). Use 8 for 8-bit; omit for FP16/BF16
quantization_method: bnb  # QLoRA via BitsAndBytes or hqq / eetq

stage: dpo
do_train: true
finetuning_type: lora
lora_rank: 8
lora_target: all
pref_beta: 0.1
pref_loss: sigmoid  # choices: [sigmoid (dpo), orpo, simpo]

# local dataset
dataset: my_ultrafeedback
dataset_dir: /mnt/cluster_storage

template: qwen
cutoff_len: 1024
max_samples: 1000
overwrite_cache: true
preprocessing_num_workers: 16

### output
output_dir: qwen2.5_7b_qlora_dpo
logging_steps: 5
save_steps: 5              # For tensorboard logging purpose too. Can increase if not using tensorboard
plot_loss: true
report_to: tensorboard  # or none

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 2
num_train_epochs: 3.0  # Low for demo purpose; adjust as needed
learning_rate: 5.0e-6
bf16: true
lr_scheduler_type: cosine
warmup_ratio: 0.1
ddp_timeout: 180000000

### ray
ray_run_name: qwen2.5_7b_qlora_dpo
ray_storage_path: /mnt/cluster_storage/
ray_num_workers: 4  # Number of GPUs to use.
resources_per_worker:
  GPU: 1
  anyscale/accelerator_shape:4xL4: 0.001  # Use this to specify a specific node shape.
  # accelerator_type:L4: 0.001            # Or use this to simply specify a GPU type.
  # See https://docs.ray.io/en/master/ray-core/accelerator-types.html#accelerator-types for a full list of accelerator types.

ray_init_kwargs:
  runtime_env:
    env_vars:
      # If using gated models like meta-llama/Llama-3.1-8B-Instruct
      # HF_TOKEN: <your_huggingface_token>
      # Enable faster downloads if hf_transfer is installed:
      HF_HUB_ENABLE_HF_TRANSFER: '1'
```

## Step 4: Train and monitor

With all configurations in place, you can launch fine-tuning or post-training in one of two ways:

### Option A: Run from a workspace (quick start)

The `USE_RAY=1` prefix tells LLaMA-Factory to run in distributed mode on the Ray cluster attached to your workspace.

In [3]:
%%bash
USE_RAY=1 llamafactory-cli train ../train-configs/dpo_qlora.yaml

[2026-02-08 09:30:36,172] [WARNING] [real_accelerator.py:209:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2026-02-08 09:30:36,173] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cpu (auto detect)
INFO 02-08 09:30:38 [__init__.py:248] No platform detected, vLLM is running on UnspecifiedPlatform
WARNING 02-08 09:30:38 [_custom_ops.py:20] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')


2026-02-08 09:30:40,431	INFO worker.py:1747 -- Connecting to existing Ray cluster at address: 10.128.5.218:6379...
2026-02-08 09:30:40,442	INFO worker.py:1918 -- Connected to Ray cluster. View the dashboard at https://session-c1mvc6t862zj4fbguuknngnrgv.i.anyscaleuserdata.com 
2026-02-08 09:30:40,443	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_50420c404e85b1830c428a285b74dd50fa30970a.zip' (0.18MiB) to Ray cluster...
2026-02-08 09:30:40,444	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_50420c404e85b1830c428a285b74dd50fa30970a.zip'.



View detailed results here: /mnt/cluster_storage/qwen2.5_7b_qlora_dpo
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2026-02-08_07-42-51_475236_185/artifacts/2026-02-08_09-30-40/qwen2.5_7b_qlora_dpo/driver_artifacts`
(TrainTrainable pid=5784, ip=10.128.7.103) [2026-02-08 09:31:45,373] [WARNING] [real_accelerator.py:209:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
(TrainTrainable pid=5784, ip=10.128.7.103) [2026-02-08 09:31:45,374] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cpu (auto detect)

Training started with configuration:
╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Training config                                                                                              │
├──────────────────────────────────────────────────────────────────────────────────────────────────

(RayTrainWorker pid=5917, ip=10.128.7.103) Setting up process group for: env:// [rank=0, world_size=4]
(TorchTrainer pid=5784, ip=10.128.7.103) Started distributed worker processes: 
(TorchTrainer pid=5784, ip=10.128.7.103) - (node_id=c0a063c3a2d8319e8a19333c768480af090e91c460068963ebf9fb27, ip=10.128.7.103, pid=5917) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=5784, ip=10.128.7.103) - (node_id=c0a063c3a2d8319e8a19333c768480af090e91c460068963ebf9fb27, ip=10.128.7.103, pid=5920) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=5784, ip=10.128.7.103) - (node_id=c0a063c3a2d8319e8a19333c768480af090e91c460068963ebf9fb27, ip=10.128.7.103, pid=5919) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=5784, ip=10.128.7.103) - (node_id=c0a063c3a2d8319e8a19333c768480af090e91c460068963ebf9fb27, ip=10.128.7.103, pid=5918) world_rank=3, local_rank=3, node_rank=0


(RayTrainWorker pid=5918, ip=10.128.7.103) [2026-02-08 09:31:57,268] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
(RayTrainWorker pid=5917, ip=10.128.7.103) [WARNING|2026-02-08 09:32:01] llamafactory.hparams.parser:148 >> We recommend enable `upcast_layernorm` in quantized training.
(RayTrainWorker pid=5917, ip=10.128.7.103) [INFO|2026-02-08 09:32:01] llamafactory.hparams.parser:143 >> Set `ddp_find_unused_parameters` to False in DDP training since LoRA is enabled.
(RayTrainWorker pid=5917, ip=10.128.7.103) [INFO|2026-02-08 09:32:01] llamafactory.hparams.parser:406 >> Process rank: 0, world size: 4, device: cuda:0, distributed training: True, compute dtype: torch.bfloat16


...

(RayTrainWorker pid=5917, ip=10.128.7.103) [INFO|configuration_utils.py:698] 2026-02-08 09:34:14,834 >> loading configuration file config.json from cache at /home/ray/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B-Instruct/snapshots/a09a35458c702b33eeacc393d103063234e8bc28/config.json
(RayTrainWorker pid=5917, ip=10.128.7.103) [INFO|configuration_utils.py:770] 2026-02-08 09:34:14,834 >> Model config Qwen2Config {
(RayTrainWorker pid=5917, ip=10.128.7.103)   "architectures": [
(RayTrainWorker pid=5917, ip=10.128.7.103)     "Qwen2ForCausalLM"
(RayTrainWorker pid=5917, ip=10.128.7.103)   ],
(RayTrainWorker pid=5917, ip=10.128.7.103)   "attention_dropout": 0.0,
(RayTrainWorker pid=5917, ip=10.128.7.103)   "bos_token_id": 151643,
(RayTrainWorker pid=5917, ip=10.128.7.103)   "eos_token_id": 151645,
(RayTrainWorker pid=5917, ip=10.128.7.103)   "hidden_act": "silu",
(RayTrainWorker pid=5917, ip=10.128.7.103)   "hidden_size": 3584,
(RayTrainWorker pid=5917, ip=10.128.7.103)   "initializer_rang

(RayTrainWorker pid=5917, ip=10.128.7.103) ***** train metrics *****
(RayTrainWorker pid=5917, ip=10.128.7.103)   epoch                    =        3.0
(RayTrainWorker pid=5917, ip=10.128.7.103)   total_flos               = 12512612GF
(RayTrainWorker pid=5917, ip=10.128.7.103)   train_loss               =     0.6595
(RayTrainWorker pid=5917, ip=10.128.7.103)   train_runtime            = 0:01:26.23
(RayTrainWorker pid=5917, ip=10.128.7.103)   train_samples_per_second =      3.479
(RayTrainWorker pid=5917, ip=10.128.7.103)   train_steps_per_second   =      0.452
(RayTrainWorker pid=5917, ip=10.128.7.103) Figure saved at: qwen2.5_7b_qlora_dpo/training_loss.png


(RayTrainWorker pid=5917, ip=10.128.7.103) [INFO|modelcard.py:450] 2026-02-08 09:34:15,309 >> Dropping the following result as it does not have all the necessary fields:
(RayTrainWorker pid=5917, ip=10.128.7.103) {'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}


(RayTrainWorker pid=5917, ip=10.128.7.103) Figure saved at: qwen2.5_7b_qlora_dpo/training_rewards_accuracies.png
(RayTrainWorker pid=5917, ip=10.128.7.103) [WARNING|2026-02-08 09:34:15] llamafactory.extras.ploting:148 >> No metric eval_loss to plot.

Training completed after 8 iterations at 2026-02-08 09:34:16. Total running time: 3min 36s


2026-02-08 09:34:16,825	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/mnt/cluster_storage/qwen2.5_7b_qlora_dpo' in 0.0941s.


### Option B: Run as an Anyscale job (production)

For longer or production runs, submit the training as an **Anyscale job**. Jobs run outside your interactive session for better stability, retries, and durable logs. Package LLaMA-Factory and other libraries in a container image and launch with a short job config. See [Run LLaMA-Factory as an Anyscale job](https://docs.anyscale.com/llm/fine-tuning/llamafactory-jobs) for the step-by-step guide.

### Tracking with TensorBoard
If you enabled TensorBoard logging (`report_to: tensorboard` in your YAML), you can watch metrics (for example, training loss) update live and compare multiple runs with the same run name side-by-side.

- **While the job is running:** LLaMA-Factory prints a ready-to-run command that starts with `tensorboard --logdir`. Open a new terminal and run it. For example:
  ```bash
  tensorboard --logdir /tmp/ray/session_*/artifacts/*/qwen2.5_7b_qlora_dpo/driver_artifacts
  ```

- **After the job:** Point TensorBoard at `{ray_storage_path}/{ray_run_name}/`. Each `TorchTrainer_*` subfolder holds event files for a single run. Using the parent folder aggregates all runs for easy comparison.
  ```bash
  tensorboard --logdir /mnt/cluster_storage/qwen2.5_7b_qlora_dpo
  ```

In your Anyscale workspace, look for the open **port 6006** labeled **TensorBoard** to view the dashboards.

![Anyscale workspace showing open ports with TensorBoard on port 6006](https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/open-ports.png)

**TensorBoard example**

![TensorBoard](https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/3.2.2/3.2.2-tensorboard.png)

For a more detailed guide on tracking experiments with other tools such as Weights & Biases or MLflow, see [Observability and tracking](https://docs.anyscale.com/llm/fine-tuning/observability-and-tracking).

## Step 5: Locate checkpoints

Ray Train writes checkpoints under `ray_storage_path/ray_run_name`. In this example run, the path is: `/mnt/cluster_storage/qwen2.5_7b_qlora_dpo`. 

Inside, you see a **trainer session** directory named like:
`TorchTrainer_ff224_00000_0_2025-09-19_15-57-20/`.

- Ray Train creates `TorchTrainer_*` **when the trainer starts**; the suffix encodes a short run ID and the **start timestamp**.
- Within that directory, Ray Train names checkpoints `checkpoint_000xxx/`, where the number is the saved ordered checkpoints.

Control the save cadence with `save_strategy` and `save_steps`. For instructions on how to resume interrupted training with `resume_from_checkpoint` and more, see [Understand the artifacts directory](https://docs.anyscale.com/llm/fine-tuning/checkpointing#artifacts-directory).

## Step 6: Export the model

If you use LoRA, you can keep the base model and adapters separate for [multi-LoRA deployment](https://docs.anyscale.com/llm/serving/multi-lora) or [merge the adapters](https://docs.anyscale.com/llm/fine-tuning/checkpointing#merge-lora) into the base model for low-latency inference. 

For full fine-tuning or freeze-tuning, export the fine-tuned model directly.

You may optionally apply [post-training quantization](https://docs.anyscale.com/llm/fine-tuning/checkpointing#ptq) on merged or full models before serving.